In [1]:
import os

base_path = '/kaggle/input/hengck23-submit-physionet'

print("Contents of base path:")
for item in os.listdir(base_path):
    full_path = os.path.join(base_path, item)
    print(f"  {item}/ " if os.path.isdir(full_path) else f"  {item}")
    
    if os.path.isdir(full_path):
        for sub_item in os.listdir(full_path):
            sub_path = os.path.join(full_path, sub_item)
            print(f"    {sub_item}/" if os.path.isdir(sub_path) else f"    {sub_item}")
            
            if os.path.isdir(sub_path):
                for sub_sub in os.listdir(sub_path)[:10]:  # Limit to first 10
                    print(f"      {sub_sub}")

Contents of base path:
  hengck23-submit-physionet/ 
    640106434-0001.overlay.png
    setup/
      numpy-2.3.4-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl
      connected_components_3d-3.26.1-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl
    stage1_model.py
    stage0_common.py
    stage1_common.py
    stage2_common.py
    stage2_model.py
    sample_list.py
    weight/
      stage0-last.checkpoint.pth
      stage1-last.checkpoint.pth
      stage2-00005810.checkpoint.pth
    stage0_model.py
    640106434-0001.gridpoint_xy.npy


In [2]:
# Advanced PhysioNet ECG Image Digitization with ARCO/RCI Denoising
# Based on @hengck23's 3-stage pipeline with validation framework
# ARCO/RCI denoising applied post-Stage 2 to refine 12-lead signals

import subprocess
import sys
import os
import math
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from fractions import Fraction
from scipy import signal as scipy_signal
import inspect
import warnings

warnings.filterwarnings('ignore')

# ===========================================
# 1. Install dependencies from hengck23's setup
# ===========================================
HENGCK23_PATH = '/kaggle/input/hengck23-submit-physionet/hengck23-submit-physionet'
SETUP_PATH = f'{HENGCK23_PATH}/setup'

print("Installing dependencies...")
try:
    subprocess.check_call([
        sys.executable, '-m', 'pip', 'install', '--quiet',
        f'{SETUP_PATH}/connected_components_3d-3.26.1-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl'
    ])
    print("Dependencies installed.")
except Exception as e:
    print(f"Dependency installation skipped or failed (might be already installed): {e}")

# Add hengck23's code to path
sys.path.insert(0, HENGCK23_PATH)

# ===========================================
# 2. Dynamic Import & Discovery
# ===========================================
print("Discovering module contents...")
import stage0_model
import stage1_model
import stage2_model
import stage0_common
import stage1_common
import stage2_common

def find_net_class(module):
    """Find the main neural network class in a module."""
    module_name = module.__name__
    candidates = []
    for name, obj in inspect.getmembers(module):
        if inspect.isclass(obj) and issubclass(obj, nn.Module) and obj is not nn.Module:
            if hasattr(obj, '__module__') and obj.__module__ == module_name:
                candidates.append((name, obj))
    
    if not candidates:
        for name, obj in inspect.getmembers(module):
            if inspect.isclass(obj) and issubclass(obj, nn.Module) and 'Net' in name:
                candidates.append((name, obj))
    
    if candidates:
        for name, obj in candidates:
            if 'Net' in name: return obj
        return candidates[0][1]
    return None

def find_function(module, possible_names):
    """Find a function by trying multiple possible names."""
    for name in possible_names:
        if hasattr(module, name): return getattr(module, name)
    return None

# Discover Net classes
Stage0Net = find_net_class(stage0_model)
Stage1Net = find_net_class(stage1_model)
Stage2Net = find_net_class(stage2_model)

# Discover helper functions
image_to_batch = find_function(stage0_common, ['image_to_batch', 'to_batch', 'make_batch'])
output_to_predict = find_function(stage0_common, ['output_to_predict', 'to_predict', 'predict'])
normalise_by_homography = find_function(stage0_common, ['normalise_by_homography', 'normalize_by_homography'])
stage1_output_to_predict = find_function(stage1_common, ['stage1_output_to_predict', 'output_to_predict'])
rectify_image = find_function(stage1_common, ['rectify_image', 'rectify', 'warp_image'])
pixel_to_series = find_function(stage2_common, ['pixel_to_series', 'to_series'])
filter_series_by_limits = find_function(stage2_common, ['filter_series_by_limits', 'filter_series'])

# Check critical components
if None in [Stage0Net, Stage1Net, Stage2Net, image_to_batch, rectify_image, pixel_to_series]:
    raise RuntimeError("Critical models or functions could not be dynamically discovered.")

# ===========================================
# 3. Configuration & Model Loading
# ===========================================
KAGGLE_DIR = '/kaggle/input/physionet-ecg-image-digitization'
WEIGHT_DIR = f'{HENGCK23_PATH}/weight'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
FLOAT_TYPE = torch.float32
LEADS = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
TYPE_IDS = ['0001', '0003', '0004', '0005', '0006', '0009', '0010', '0011', '0012']
BASE_FS = 500

def load_net(model, path):
    checkpoint = torch.load(path, map_location='cpu')
    state_dict = checkpoint.get('state_dict', checkpoint.get('model_state_dict', checkpoint))
    model.load_state_dict(state_dict, strict=False)
    return model

print('Loading models...')
stage0_net = load_net(Stage0Net(pretrained=False), f'{WEIGHT_DIR}/stage0-last.checkpoint.pth').to(DEVICE).eval()
stage1_net = load_net(Stage1Net(pretrained=False), f'{WEIGHT_DIR}/stage1-last.checkpoint.pth').to(DEVICE).eval()
stage2_net = load_net(Stage2Net(pretrained=False), f'{WEIGHT_DIR}/stage2-00005810.checkpoint.pth').to(DEVICE).eval()
print('Models loaded successfully!')

# ===========================================
# 4. ARCO/RCI Denoising Implementation
# ===========================================
def make_rational_frequencies(max_q: int = 30) -> list:
    arcs = set()
    for q in range(2, max_q + 1):
        for a in range(1, q):
            if math.gcd(a, q) == 1:
                f = Fraction(a, q)
                if f > Fraction(1, 2): f = 1 - f
                arcs.add(f)
    return sorted(arcs, key=float)

def _bin_frequencies(n: int, device):
    return torch.arange(0, n // 2 + 1, device=device, dtype=torch.float32) / float(n)

def gaussian_arc_weights(n: int, centers, q_vals, sigma_factor=0.5):
    device = centers.device
    freqs = _bin_frequencies(n, device)
    sigma = sigma_factor / (q_vals ** 2 + 1e-6)
    diff = freqs[None, :] - centers[:, None]
    W = torch.exp(-0.5 * (diff / (sigma[:, None] + 1e-9)) ** 2)
    if W.shape[1] > 0: W[:, 0] = 0.0
    return W / (W.sum(dim=0, keepdim=True) + 1e-9)

def power_spectrum(x, use_hann=True):
    x = x.float()
    B, T = x.shape
    if use_hann:
        w = torch.hann_window(T, device=x.device, dtype=torch.float32).unsqueeze(0)
        xw = x * w
    else:
        xw = x
    X = torch.fft.rfft(xw, dim=1)
    P = (X.real ** 2 + X.imag ** 2)
    return P / (P.sum(dim=1, keepdim=True) + 1e-12)

class ArcRCI:
    def __init__(self, max_q=30, sigma_factor=0.5, device='cpu'):
        fracs = make_rational_frequencies(max_q)
        self.centers = torch.tensor([float(f) for f in fracs], dtype=torch.float32, device=device)
        self.q_vals = torch.tensor([f.denominator for f in fracs], dtype=torch.float32, device=device)
        self.sigma_factor = sigma_factor

    def rci(self, x, use_hann=True):
        P = power_spectrum(x, use_hann)
        W = gaussian_arc_weights(x.shape[1], self.centers, self.q_vals, self.sigma_factor)
        arcogram = torch.matmul(P, W.t())
        return torch.clamp(arcogram.sum(dim=1), 0.0, 1.0)

def denoised_multi_lead_arco(signals_dict, fs=500, window_size=200, step=50, max_q=30):
    lead_names = list(signals_dict.keys())
    signals_list = [signals_dict[name] for name in lead_names]
    L_max = max(len(s) for s in signals_list)
    sigs = np.stack([np.pad(s, (0, L_max - len(s)), mode='edge') if len(s) < L_max else s[:L_max] for s in signals_list], axis=0).astype(np.float32)
    
    n_leads, n_samples = sigs.shape
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    arc = ArcRCI(max_q, device=device)
    
    kern_size = min(window_size // 4, 51)
    kern = np.ones(kern_size) / kern_size
    smooth_signals = np.array([np.convolve(lead, kern, mode='same') for lead in sigs])
    refined = np.zeros_like(sigs)
    b, a = scipy_signal.butter(4, 40 / (fs / 2), 'low')
    
    for i in range(n_leads):
        lead = sigs[i]
        lead_t = torch.from_numpy(lead).unsqueeze(0).to(device)
        rci_profile = []
        end = 0
        for start in range(0, n_samples - window_size + 1, step):
            end = start + window_size
            rci = arc.rci(lead_t[:, start:end]).item()
            rci_profile.append(rci)
            denom = max(rci_profile) if rci_profile else 1.0
            w = min(1.0, rci / denom) if denom > 0 else 0.5
            refined[i, start:end] = w * lead[start:end] + (1 - w) * smooth_signals[i, start:end]
        
        if end < n_samples:
            last_w = min(1.0, rci_profile[-1] / max(rci_profile)) if rci_profile else 0.5
            refined[i, end:] = last_w * lead[end:] + (1 - last_w) * smooth_signals[i, end:]
        
        refined[i] = scipy_signal.filtfilt(b, a, refined[i])
    
    return {name: refined[i, :len(signals_dict[name])] for i, name in enumerate(lead_names)}

# ===========================================
# 5. Processing Functions (With Fixes)
# ===========================================
def add_dummy_targets_stage0(batch, device):
    B, C, H, W = batch['image'].shape
    if 'marker' not in batch: batch['marker'] = torch.zeros((B, H, W), device=device, dtype=torch.long)
    if 'orientation' not in batch: batch['orientation'] = torch.zeros((B,), device=device, dtype=torch.float32)
    return batch

def add_dummy_targets_stage1(batch, device):
    B, C, H, W = batch['image'].shape
    if 'marker' not in batch: batch['marker'] = torch.zeros((B, H, W), device=device, dtype=torch.long)
    if 'gridpoint' not in batch: batch['gridpoint'] = torch.zeros((B, 1, H, W), device=device, dtype=torch.float32)
    for k in ['gridhline', 'gridvline']:
        if k not in batch: batch[k] = torch.zeros((B, H, W), device=device, dtype=torch.long)
    return batch

def add_dummy_targets_stage2(batch, device):
    B, C, H, W = batch['image'].shape
    if 'pixel' not in batch: batch['pixel'] = torch.zeros((B, 4, H, W), device=device, dtype=torch.float32)
    return batch

def process_stage0(image, stage0_net, device='cuda', float_type=None):
    batch = image_to_batch(image)
    batch['image'] = batch['image'].to(device)
    batch = add_dummy_targets_stage0(batch, device) 
    
    with torch.amp.autocast('cuda', dtype=float_type):
        with torch.no_grad():
            output = stage0_net(batch)
            rotated, keypoint = output_to_predict(image, batch, output)
            normalised, keypoint, homography = normalise_by_homography(rotated, keypoint)
    torch.cuda.empty_cache()
    return normalised, keypoint, homography

def process_stage1(normalised, stage1_net, device='cuda', float_type=None, num_tta=4):
    batch = {'image': torch.from_numpy(np.ascontiguousarray(normalised.transpose(2, 0, 1))).unsqueeze(0)}
    batch = add_dummy_targets_stage1(batch, device)
    
    crop = batch['image'].clone().byte().to(device)
    trial_batch = {'image': crop}
    trial_batch = add_dummy_targets_stage1(trial_batch, device)
    
    with torch.amp.autocast('cuda', dtype=float_type):
        with torch.no_grad():
            output = stage1_net(trial_batch)
            gridpoint_xy, more = stage1_output_to_predict(normalised, batch, output)
            rectified = rectify_image(normalised, gridpoint_xy)
    torch.cuda.empty_cache()
    return rectified, gridpoint_xy

def process_stage2(rectified, signal_length, stage2_net, device='cuda', float_type=None):
    x0, x1, y0, y1 = 0, 2176, 0, 1696
    zero_mv = [703.5, 987.5, 1271.5, 1531.5]
    mv_to_pixel = 79.0
    t0, t1 = 118, 2080
    
    crop = rectified[y0:y1, x0:x1]
    batch = {'image': torch.from_numpy(np.ascontiguousarray(crop.transpose(2, 0, 1))).unsqueeze(0)}
    batch['image'] = batch['image'].to(device)
    batch = add_dummy_targets_stage2(batch, device)
    
    with torch.amp.autocast('cuda', dtype=float_type):
        with torch.no_grad():
            output = stage2_net(batch)
            pixel = output['pixel'].data.cpu().numpy()[0]
            
    series_in_pixel = pixel_to_series(pixel[..., t0:t1], zero_mv, signal_length)
    series = (np.array(zero_mv).reshape(4, 1) - series_in_pixel) / mv_to_pixel
    series = filter_series_by_limits(series)
    torch.cuda.empty_cache()
    return series

def extract_leads_from_series(series):
    """Standard extraction from 4-row series."""
    length = series.shape[1]
    quarter = length // 4
    return {
        'I': series[0, 0:quarter], 'aVR': series[0, quarter:2*quarter], 'V1': series[0, 2*quarter:3*quarter], 'V4': series[0, 3*quarter:],
        'II': series[3, :], 'aVL': series[1, quarter:2*quarter], 'V2': series[1, 2*quarter:3*quarter], 'V5': series[1, 3*quarter:],
        'III': series[2, 0:quarter], 'aVF': series[2, quarter:2*quarter], 'V3': series[2, 2*quarter:3*quarter], 'V6': series[2, 3*quarter:]
    }

# ===========================================
# 6. Main Pipeline (Integrated)
# ===========================================
def process_image_pipeline(image, lead_records, stage0_net, stage1_net, stage2_net, use_arco=True):
    # Data Integrity: Ensure lead_records works for both schema types
    # If we have the "compact" schema (fs, sig_len), we don't filter by lead='II'
    
    is_detailed_schema = 'lead' in lead_records.columns
    
    # Stage 0
    normalised, _, _ = process_stage0(image, stage0_net, DEVICE, FLOAT_TYPE)
    # Stage 1
    rectified, _ = process_stage1(normalised, stage1_net, DEVICE, FLOAT_TYPE)
    
    # Stage 2 Prep: Determine Signal Length
    signal_length = 5000 # Default fallback
    
    if is_detailed_schema:
        lead_i_record = lead_records[lead_records['lead'] == 'II']
        if not lead_i_record.empty:
            signal_length = int(lead_i_record.iloc[0]['number_of_rows'])
    else:
        # Compact schema handling: use 'sig_len' if available
        if 'sig_len' in lead_records.columns:
            signal_length = int(lead_records.iloc[0]['sig_len'])
    
    # Stage 2
    series = process_stage2(rectified, signal_length, stage2_net, DEVICE, FLOAT_TYPE)
    predicted_leads = extract_leads_from_series(series)
    
    # ARCO/RCI Denoising
    if use_arco:
        predicted_leads = denoised_multi_lead_arco(predicted_leads, fs=BASE_FS)
        
    return predicted_leads, normalised, rectified

# ===========================================
# 7. Helpers for Submission/Scoring
# ===========================================
def build_submission_rows(predicted_leads, image_id, lead_records):
    rows = []
    
    # Handle both schemas
    if 'lead' in lead_records.columns:
        # Detailed schema (one row per lead)
        iterator = lead_records.iterrows()
    else:
        # Compact schema: Create synthetic iterator for 12 leads
        # sig_len might be in the dataframe
        sig_len = int(lead_records.iloc[0]['sig_len']) if 'sig_len' in lead_records.columns else 5000
        # Create a generator of (index, row-like dict)
        synthetic_data = []
        for lead in LEADS:
            synthetic_data.append({'lead': lead, 'number_of_rows': sig_len})
        iterator = enumerate(synthetic_data) # enumerating list of dicts acts like iterrows somewhat

    for _, row in iterator:
        # Access keys safely (row can be Series or dict)
        lead = row['lead']
        num_rows = int(row['number_of_rows'])
        
        sig = predicted_leads.get(lead, np.zeros(num_rows))
        
        if len(sig) != num_rows:
            sig = np.interp(np.linspace(0, 1, num_rows), np.linspace(0, 1, len(sig)), sig)
            
        for r in range(num_rows):
            rows.append({'id': f"{image_id}_{r}_{lead}", 'value': float(sig[r])})
    return rows

def load_ground_truth(image_id, kaggle_dir):
    path = f'{kaggle_dir}/train/{image_id}/{image_id}.csv'
    if not os.path.exists(path): return {}
    df = pd.read_csv(path)
    return {lead: df[lead].ffill().bfill().values for lead in LEADS if lead in df.columns}

def score(sol_df, sub_df):
    merged = sol_df.merge(sub_df, on='id', suffixes=('_true', '_pred'))
    t, p = merged['value_true'].values, merged['value_pred'].values
    noise_var = np.var(t - p)
    return 10 * np.log10(np.var(t) / noise_var) if noise_var > 0 else 0.0

# ===========================================
# 8. Inference & Validation Loops
# ===========================================
print('\n' + '='*40 + '\nINFERENCE\n' + '='*40)
test_path = f'{KAGGLE_DIR}/test.csv'
if os.path.exists(test_path):
    test_df = pd.read_csv(test_path)
    test_df['id'] = test_df['id'].astype(str)
    image_ids = test_df['id'].unique().tolist()
    submission_rows = []
    
    for n, image_id in enumerate(image_ids):
        print(f'\rProcessing {n+1}/{len(image_ids)}: {image_id}', end='', flush=True)
        filename = f'{KAGGLE_DIR}/test/{image_id}.png'
        if not os.path.exists(filename): continue
        
        try:
            image = cv2.imread(filename, cv2.IMREAD_COLOR_RGB)
            lead_records = test_df[test_df['id'] == image_id].copy()
            predicted_leads, _, _ = process_image_pipeline(image, lead_records, stage0_net, stage1_net, stage2_net)
            if predicted_leads:
                submission_rows.extend(build_submission_rows(predicted_leads, image_id, lead_records))
        except Exception as e:
            print(f" Error: {e}")

    if submission_rows:
        pd.DataFrame(submission_rows).to_csv('submission.csv', index=False)
        print('\nSubmission saved to submission.csv')

print('\n' + '='*40 + '\nVALIDATION\n' + '='*40)
train_path = f'{KAGGLE_DIR}/train.csv'
if not os.getenv('KAGGLE_IS_COMPETITION_RERUN') and os.path.exists(train_path):
    train_df = pd.read_csv(train_path)
    
    # --- ADAPTER: Handle Compact Schema ---
    if 'lead' not in train_df.columns and 'sig_len' in train_df.columns:
        print("Notice: Detected 'compact' train.csv schema (['id', 'fs', 'sig_len']). Using adapter.")
        # We don't need to explode the dataframe here, we just need to ensure
        # process_image_pipeline and build_submission_rows handle the missing 'lead' column gracefully
        # which we updated above.
    elif 'lead' not in train_df.columns:
        print(f"FATAL ERROR: Unknown schema. Columns found: {list(train_df.columns)}")
        # Stop validation if schema is completely unknown
        train_df = pd.DataFrame() 

    if not train_df.empty:
        train_df['id'] = train_df['id'].astype(str)
        val_ids = train_df['id'].unique()[:5].tolist() 
        scores = []
        
        for image_id in val_ids:
            lead_records = train_df[train_df['id'] == image_id].copy()
            if lead_records.empty: continue
            
            found_file = False
            for type_id in TYPE_IDS:
                filename = f'{KAGGLE_DIR}/train/{image_id}/{image_id}-{type_id}.png'
                if os.path.exists(filename):
                    print(f"Validating {image_id} ({type_id})...")
                    image = cv2.imread(filename, cv2.IMREAD_COLOR_RGB)
                    
                    # Pipeline handles schema differences internally now
                    preds, _, _ = process_image_pipeline(image, lead_records, stage0_net, stage1_net, stage2_net)
                    
                    if preds:
                        gt = load_ground_truth(image_id, KAGGLE_DIR)
                        # We need a 'lead_records' that works for building rows. 
                        # If we have the compact schema, we need to pass a compliant structure or let the function handle it.
                        # The updated build_submission_rows handles the compact schema by checking columns.
                        
                        # Create solution rows (Ground Truth)
                        # Ground truth loading might yield different keys than preds if file is partial
                        sol = pd.DataFrame(build_submission_rows(gt, image_id, lead_records))
                        
                        # Create submission rows (Prediction)
                        sub = pd.DataFrame(build_submission_rows(preds, image_id, lead_records))
                        
                        if not sol.empty and not sub.empty:
                            s = score(sol, sub)
                            scores.append(s)
                            print(f" -> SNR: {s:.2f} dB")
                    found_file = True
                    break
            if not found_file: print(f"No image found for {image_id}")

        if scores: print(f"\nAverage Validation SNR: {np.mean(scores):.2f} dB")

print('\nDone!')

Installing dependencies...
Dependencies installed.
Discovering module contents...
THIS_DIR: /kaggle/input/hengck23-submit-physionet/hengck23-submit-physionet
REF_PT: (9, 2)
/kaggle/input/hengck23-submit-physionet/hengck23-submit-physionet
/kaggle/input/hengck23-submit-physionet/hengck23-submit-physionet
Loading models...
Models loaded successfully!

INFERENCE
Processing 2/2: 2352854581
Submission saved to submission.csv

VALIDATION
Notice: Detected 'compact' train.csv schema (['id', 'fs', 'sig_len']). Using adapter.
Validating 7663343 (0001)...
 -> SNR: -4.29 dB
Validating 10140238 (0001)...
 -> SNR: -3.78 dB
Validating 11842146 (0001)...
 -> SNR: -4.40 dB
Validating 19030958 (0001)...
 -> SNR: -3.71 dB
Validating 19585145 (0001)...
 -> SNR: -4.68 dB

Average Validation SNR: -4.17 dB

Done!
